In [1]:
def get_values(widget):
    out_val = widget.value
    return out_val

def adv_values():
    # more values setting
    max_depth = [350, 1500]
    rock_rho = 2.7
    CN_lambda = 160
    mu = rock_rho / CN_lambda
    dt = 1

    # He diffusion inputs
    rho = 2.65 # quartz density
    nx = 100
    r = gr_sz.value
    dx = (r / nx)
    x = np.arange(dx/2, r, dx)
    x_up = x + (dx/2)
    x_low = x - (dx/2)
    x_vol = [(4/3 * np.pi * x_up[i]**3) - (4/3 * np.pi * x_low[i]**3) for i in range(len(x))]
    shell_mass = [(x_vol[i] * rho) for i in range(len(x_vol))]

    # Bedrock thermal properties
    alpha = 1.5e-6
    
    P0_Be10 = SLHL_Be10.value * scaling_factor.value
    P0_He3 = SLHL_He3.value * scaling_factor.value
    
    adv_list = [max_depth, rock_rho, CN_lambda, mu, dt, rho, nx, r, dx, x, x_up, x_low, x_vol, shell_mass, alpha, P0_Be10, P0_He3] 
    
    return adv_list

def set_values():
    var_list = [erosion_type, total_time, SLHL_C14, SLHL_Be10, SLHL_He3,
                scaling_factor, gr_sz, MAT, T_AMP, save_output]
    set_list = [get_values(var_list[i]) for i in range(len(var_list))]
    
    more_values = adv_values()
    
    if set_list[0] == 'stochastic':
        set_list.append(get_values(scenarios))
        set_list.append(get_values(stoch_base_ER))
        set_list.append(get_values(sample_no))
        set_list.append(get_values(pareto_c))
        set_list.extend(more_values)
    
    if set_list[0] =='constant':
        set_list.append(get_values(erosion_rate))
        set_list.append(get_values(shift_ER))
        set_list.append(get_values(time_ER_shift))
        set_list.append(get_values(new_ER) / get_values(erosion_rate))
        set_list.extend(more_values)
    
    if set_list[0] == 'none':
        set_list = set_list
        
    return set_list

def post_process(CRN_output, He3_output):
    ERs_all = He3_output[2]
    export_inputs()
    
    He3_percent_mat = np.empty((total_time, scenarios))
    
    for i in range(scenarios):
        He3_percent_mat[:,i] = np.divide(He3_output[1][:,i], CRN_output[1][:,1,i]) * P0_Be10 / P0_He3

    master_array = np.empty((total_time, 11, scenarios))

    for i in range(scenarios):
        master_array[:,:3,i] = CRN_output[1][:,:,i]
        master_array[:,3,i] = He3_output[1][:,i]
        master_array[:,4,i] = He3_percent_mat[:,i]
        master_array[:,5,i] = rockfall_output[:,i]
        master_array[:,6:,i] = comparison_constants[:,:,i]
        
        np.savetxt('samples_output_' + str(ERs_all[i]) + '.csv',
               samples[:,:,i],
               delimiter = ',')
        
        np.savetxt('comparison_output_' + str(ERs_all[i]) + '.csv',  
               master_array[:,:,i],
               delimiter = ',')
        
    np.savetxt('measured_output_' + str(np.mean(ERs_all)) + '.csv',
              measured,
              delimiter = ',')
    
    master_means = [[np.mean(master_array[2500:,i,j]) for j in range(scenarios)] for i in range(11)]
    
    np.savetxt('all_year_means_output_' + str(np.mean(ERs_all)) + '.csv',
              master_means,
              delimiter = ',')
    
    master_df = [pd.DataFrame(data = master_array[:,:,i],
                index = np.arange(0,total_time,1),
                columns = ['C-14 Conc. Stoch', 
                           'Be-10 Conc. Stoch', 
                           'C-14/Be-10 Ratio Stoch',
                           'He-3 Conc. Stoch',
                           'Perc. He-3 Stoch',
                           'Rockfall Size',
                           'C-14 Conc. Constant',
                           'Be-10 Conc. Constant',
                           'C-14/Be-10 Ratio Constant',
                           'He-3Conc. Constant',
                           'Perc. He-3 Constant']) for i in range(scenarios)]
   
    return master_array, master_df, master_means

def export_inputs():
    input_export = ['Erosion Type: ', str(erosion_type), '\n', 
                     '\n Total Time: ', str(total_time),
                     '\n SLHL C14 P0: ', str(SLHL_C14), 
                     '\n SLHL Be10 P0: ', str(SLHL_Be10),
                     '\n SLHL He3 P0: ', str(SLHL_He3), 
                     '\n CN Scaling Factor: ', str(scaling_factor), 
                     '\n Grain Size: ', str(gr_sz),
                     '\n Mean Annual T (C): ', str(MAT),
                     '\n Seasonal Temperature Amplitude (C): ', str(T_AMP),
                     '\n Number of Scenarios: ', str(scenarios),
                     '\n Base (Mean) Erosion Rate: ', str(stoch_base_ER), 
                     '\n Number of Samples: ', str(sample_no),
                     '\n Max Depth for Calculations: ', str(max_depth), 
                     '\n Rock Density (g cm3) ', str(rock_rho), 
                     '\n Cosmogenic Nuclide Atten. Length: ', str(CN_lambda)]
    var_export = open('STEIN_variables_STO_' + str(np.mean(measured_ER)) + '_' + str(time.strftime('%d-%m-%Y_%H.%M')[:18]) + '.txt', 'w')   

    var_export.writelines(input_export)
    var_export.close()
    
def export_var_function():
    all_variables = [erosion_type, rockfalls_imported[1], total_time, SLHL_C14, SLHL_Be10, SLHL_He3, scaling_factor, gr_sz, 
                     MAT, T_AMP, save_output, scenarios, stoch_base_ER, sample_no, pareto_c, max_depth, rock_rho, 
                     CN_lambda, mu, dt, rho, nx, r, dx, x, x_up, x_low, x_vol, shell_mass, alpha, P0_Be10, P0_He3]
    variable_labels = ['Erosion Type', 'Erosion Rates', 'Total Time', 'SLHL C14 P0', 'SLHL Be10 P0', 'SLHL He3 P0', 'CN Scaling Factor',
                    'Grain Size', 'Mean Annual T (C)', 'Seasonal T Amplitude (C)', 'Save Outputs? (Y/N)', 'Number of Scenarios',
                    'Base Erosion Rate', 'Number of Subsamples','Pareto Shape Factor (c)', 'Maximum Depths',
                    'Rock Density', 'CN Lambda', 'CN mu', 'Time Interval for He3','Quartz Density','Nodes for He3 Solution',
                    'Grain Radius','Grain Node Spacing','Shell locations','Outer Shell Location', 'Inner Shell Location',
                    'Volume of Shell', 'Mass of Shell', 'Quartz Diffusivity', 'Be10P0', 'He3P0']
    variable_exp = pd.DataFrame(all_variables, index = variable_labels)
    
    variable_exp.to_csv('variable_values_' + str(np.mean(measured_ER)) + '_' + str(time.strftime('%d-%m-%Y_%H.%M')[:18]) + '.csv',
                          sep = ',')
    
    return variable_exp